# [★★★★★] Integrating Preprocessing with Python [WIP]

Minimum version: v2.4

In this example, we demonstrate how to use python to preprocess data by creating a frame structure. This example requires some Python familiarity. However, as we are developing a frame structure which contains mainly frame elements, the procedure is relatively straightforward.

## Prerequisites

We are going to find section properties from AISC table, which is available online.

In [ ]:
import pandas

section_table = pandas.read_excel(
    'https://www.aisc.org/globalassets/aisc/manual/v15.0-shapes-database/aisc-shapes-database-v15.0.xlsx',
    sheet_name=1,
    usecols='A:CF')
print(section_table.head())

Section properties such as area and moment of inertia can be extracted from this table using section designations. It is possible to define such a function now.

In [ ]:
def from_table(designation: str):
    index = section_table.index[section_table['AISC_Manual_Label'] == designation].tolist()
    assert len(index) == 1
    a = section_table.at[index[0], 'A'] # area
    sx = section_table.at[index[0], 'Sx'] # elastic modulus
    ix = section_table.at[index[0], 'Ix'] # moment of inertia
    zx = section_table.at[index[0], 'Zx'] # plastic modulus
    return a, sx, ix, zx

Then we can use this function to extract section properties from the table. For example,

In [ ]:
print(from_table('W44X230'))

## Geometry of the Frame Structure

For simplicity, we assume the frame structure has the same column/beam section for all the columns/beams on the same floor. Under such a condition, two lists of section designations can be provided so that elements can be created. Similarly, geometry information such as floor height, bay span, as well as floor mass, can be provided in the same way.

For example, we can define several lists as follows.

In [ ]:
girder = ['W21X68', 'W21X68', 'W21X68']  # floor 1 2 3
column = ['W14X193', 'W14X159', 'W14X159']  # column 1 2 3
mass = [30., 30., 30.]  # floor 1 2 3
span = [240., 240.]  # bay span 1 2
height = [120., 120., 120.]  # floor height 1 2 3

### Generate Node Grid

Given that span and height are given, it is possible to calculate the absolute position of the nodes.

In [ ]:
import numpy as np

x_coor = np.cumsum(span)
y_coor = np.cumsum(height)
x_coor = np.insert(x_coor, 0, 0.)
y_coor = np.insert(y_coor, 0, 0.)
print(x_coor, y_coor)

del span, height

By using the above coordinates, we can create a grid of nodes. A simple `Node` class is defined to represent nodes.

In [ ]:
from dataclasses import dataclass

@dataclass
class Node:
    tag: int
    x: float
    y: float

    def write_to(self, f_handler):
        f_handler.write(f'node {self.tag} {self.x:.2f} {self.y:.2f}\n')

node_pool: dict = {}

node_grid = np.zeros((len(y_coor), len(x_coor))).astype(int)

node_tag = 1
with open('node.sp', 'w') as f:
    for i in range(len(x_coor)):
        for j in range(len(y_coor)):
            node = Node(node_tag, x_coor[i], y_coor[j])
            node.write_to(f)
            node_pool[node_tag] = node
            node_grid[j, i] = node_tag
            node_tag += 1
del node_tag

with open('node.sp', 'r') as f:
    print(f.read())

The `node_grid` can be used to generate elements.

### Generate Beam Elements

Starting with the second row, beam elements can be generated by looping over each row.

In [ ]:
from math import sqrt

e = 29  # mpsi, Young's modulus
fy = .05  # mpsi, yield stress
hardening = .01 # isotropic hardening ratio
nm_size = 1.15 # initial surface size

@dataclass
class Section:
    tag: int
    name: str
    fp: float
    mp: float
    ea: float
    ei: float
    mult: float

    def __init__(self, tag, name):
        self.tag = tag
        self.name = name
        _a, _sx, _ix, _zx = from_table(name)
        self.fp = fy * _a
        self.mp = fy * _sx
        self.ea = e * _a
        self.ei = e * _ix
        self.mult = fy * _zx

    def write_to(self, f_handler, d, h):
        f_handler.write(
            f'section NM2D3 {self.tag} {self.ea:.4e} {self.ei:.4e} ' # basic section properties
            f'{self.fp:.4e} {self.mp:.4e} {d} ' # yield forces
            f'{h} 0. 0. {self.mult/self.mp - 1:.4f} 1. ' # hardening configurations
            f'0.\n') # linear density

@dataclass
class Element:
    tag: int
    node_i: Node
    node_j: Node
    section: Section
    element_length: float

    def __init__(self, tag, _node_i, _node_j, _section):
        self.tag = tag
        self.node_i = _node_i
        self.node_j = _node_j
        self.section = _section
        self.element_length = sqrt((_node_i.x - _node_j.x) ** 2 + (_node_i.y - _node_j.y) ** 2)

    def write_to(self, f_handler):
        f_handler.write(f'element NMB21 {self.tag} {self.node_i.tag} {self.node_j.tag} {self.section.tag}\n')

element_tag = 1
section_tag = 1
with open('beam.sp', 'w') as f:
    for i in range(1, len(y_coor)):
        section = Section(section_tag, girder[i - 1])
        section.write_to(f, nm_size, hardening)
        for j in range(len(x_coor) - 1):
            node_i = node_grid[i, j]
            node_j = node_grid[i, j + 1]
            element = Element(element_tag, node_pool[node_i], node_pool[node_j], section)
            element.write_to(f)
            element_tag += 1
        section_tag += 1

with open('beam.sp', 'r') as f:
    print(f.read())

### Generate Column Elements

Similarly, column elements can be generated by looping over each column.

In [ ]:
with open('column.sp', 'w') as f:
    for i in range(len(y_coor) - 1):
        section = Section(section_tag, column[i])
        section.write_to(f, nm_size, hardening)
        for j in range(len(x_coor)):
            node_i = node_grid[i, j]
            node_j = node_grid[i + 1, j]
            element = Element(element_tag, node_pool[node_i], node_pool[node_j], section)
            element.write_to(f)
            element_tag += 1
        section_tag += 1

with open('column.sp', 'r') as f:
    print(f.read())

Finally, let's pack everything into an archive so that it can be [downloaded](three-storey-frame.zip).

In [ ]:
import os
from zipfile import ZipFile

file_list = ['node.sp', 'beam.sp', 'column.sp']

with ZipFile('three-storey-frame.zip', 'w') as f:
    for file in file_list:
        f.write(file)
        os.remove(file)